#아래의 코드는 '박이삭'회원님의 코드공유를 참고하여 만들었습니다. 전반적인 코드흐름은 동일하며 보다 정확한 결과값을 위해 성능이 좋은 3개의 분류모델을 찾고, 3개의 결과를 토대로 최종 결과를 도출해내었습니다.

기본 모듈 및 측정 모델 셋팅

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split

데이터 로드

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/dacon/movie_review_analysis/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dacon/movie_review_analysis/test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/dacon/movie_review_analysis/sample_submission.csv')

max_features = 15800
Accuracy 0.875

Vectorizer를 사용하였고 max_features는 1부터 20000까지 100단위로 찾아본 결과 15800에서 정확도 0.875의 가장 높은 정확도를 가졌습니다.

In [ ]:
 vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=15800)


X = train['document']
y = np.array(train.label)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=2)

X_train = vectorizer.fit_transform(X_train)
X_eval = vectorizer.transform(X_eval)

SVM = svm.SVC().fit(X_train, y_train)
KNN = KNeighborsClassifier(n_neighbors = 47).fit(X_train, y_train)
LR = LogisticRegression(class_weight='balanced', max_iter=1000).fit(X_train, y_train)
RFC = RandomForestClassifier(class_weight='balanced', n_estimators=1000).fit(X_train, y_train)
GBC = GradientBoostingClassifier(n_estimators=1000).fit(X_train, y_train)
XGB = XGBClassifier(class_weight='balanced', n_estimators=1000).fit(X_train, y_train)
LGBM = LGBMClassifier(class_weight='balanced', n_estimators=1000).fit(X_train, y_train)


print(f"max_features: {15800}, SVM: {SVM.score(X_eval, y_eval)}, KNN: {KNN.score(X_eval, y_eval)} LR: {LR.score(X_eval, y_eval)}, RFC: {RFC.score(X_eval, y_eval)}, GBC: {GBC.score(X_eval, y_eval)}, XGB: {XGB.score(X_eval, y_eval)}, LGBM: {LGBM.score(X_eval, y_eval)}" )

max_features: 15800, SVM: 0.877, KNN: 0.846 LR: 0.875, RFC: 0.826, GBC: 0.857, XGB: 0.846, LGBM: 0.846


위의 여러 모델 결과를 보면 SupportVectorMachine이 가장 성능이 좋았고 그다음 Linear regressor, GradientBoostClassifier모델이 순위를 이었습니다.
`여기서 핵심 개념은 3개의 예측값 중 1이 2개이상 나오면 1, 0이 2개이상 나오면 0으로 결과값을 도출합니다.`


In [ ]:
X = train['document']
y = np.array(train.label)

X_train = vectorizer.fit_transform(X)
X_test = vectorizer.transform(test['document'])

SVM = svm.SVC().fit(X_train, y)
LR = LogisticRegression(class_weight='balanced', max_iter=1000).fit(X_train, y)
# RFC = RandomForestClassifier(n_estimators=1000).fit(X_train, y)
GBC = GradientBoostingClassifier(n_estimators=1000).fit(X_train, y)
# XGB = XGBClassifier(n_estimators=1000).fit(X_train, y)
# LGBM = LGBMClassifier(n_estimators=1000).fit(X_train, y)

SVM_pred = SVM.predict(X_test)
LR_pred = LR.predict(X_test)
# RFC_pred = RFC.predict(X_test)
GBC_pred = GBC.predict(X_test)
# XGB_pred = XGB.predict(X_test)
# LGBM_pred = LGBM.predict(X_test) 

pred_list = [SVM_pred,
             LR_pred,
             GBC_pred]

result = []
for i in range(5000):
    count = 0
    for pred in pred_list:
        count += pred[i]

    if count >= 2: # 1이 더 많으면  
        result.append(1)
    else:
        result.append(0)

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/dacon/movie_review_analysis/submission.csv')
sample_submission.loc[:, 'label'] = result

sample_submission.to_csv('/content/drive/MyDrive/dacon/movie_review_analysis/submission.csv', index=False)

기존 public score 0.8575에서 3개의 모델을 사용한 결과 0.8615까지 성능이 향상되었습니다.  
아직 부족한 실력으로 여기까지가 한계인 것 같습니다. 추후 순위권 분들의 코드공유를 보고 공부하도록 하겠습니다.  
감사합니다!

